# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

# Given the above, I have elected to make a model to predict incremental revenue earned per offer per customer
## There are two problems:
1.  Starbucks wants to know the most effective offers per demographic.
2.  Starbucks wants to make sure they only send offers to those who are likely to purchase more products due to the offer.

## I seek to resolve these two problems by comparing customer spending per hour while under the influence of a given offer to the spending of the same customer per hour while not under the influence of any offers.

## First, I import my packages and read in data files

In [2]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Next, I examine the original files
<br> 
### Portfolio

In [3]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [4]:
portfolio.shape

(10, 6)

### Profile

In [5]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [6]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [7]:
profile.shape

(17000, 5)

In [8]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
age                 17000 non-null int64
became_member_on    17000 non-null int64
gender              14825 non-null object
id                  17000 non-null object
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.1+ KB


### Transcript

In [9]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [9]:
transcript[transcript['person'] == '4acef5f30b3c468facf21c59294d4285']

,event,person,time,value
2751,offer received,4acef5f30b3c468facf21c59294d4285,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
77703,offer viewed,4acef5f30b3c468facf21c59294d4285,192,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
104048,transaction,4acef5f30b3c468facf21c59294d4285,294,{'amount': 10.95}
113588,offer received,4acef5f30b3c468facf21c59294d4285,336,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
130735,offer viewed,4acef5f30b3c468facf21c59294d4285,348,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
133313,transaction,4acef5f30b3c468facf21c59294d4285,354,{'amount': 30.23}
141995,transaction,4acef5f30b3c468facf21c59294d4285,378,{'amount': 25.71}
145800,transaction,4acef5f30b3c468facf21c59294d4285,390,{'amount': 20.38}
147592,transaction,4acef5f30b3c468facf21c59294d4285,396,{'amount': 17.9}
204326,offer received,4acef5f30b3c468facf21c59294d4285,504,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}


#### Here, I get the offer_id information into its own column

In [10]:
def find_offer_or_amount(x):
    if next(iter(x)) == 'offer id':
        return 'offer'
    if next(iter(x)) == 'offer_id':
        return 'offer'
    if next(iter(x)) == 'amount':
        return 'amount'
    else:
        return np.nan

In [11]:
transcript['offer_or_amount'] = transcript['value'].apply(find_offer_or_amount)
transcript.head()

,event,person,time,value,offer_or_amount
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},offer
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},offer
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},offer
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},offer


#### Next, I parse out the values in 'value' into their own columns, and I drop the duplicate 'offer_id' column that is created because my prior version column accounted for both "offer id" and "offer_id"

In [12]:
transcript=pd.concat([transcript, transcript['value'].apply(pd.Series)], axis=1)
transcript.head()

,event,person,time,value,offer_or_amount,offer id,amount,offer_id,reward
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},offer,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},offer,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},offer,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},offer,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [13]:
transcript.drop(['offer_id'],axis=1,inplace=True)
transcript = transcript.rename(index = str, columns = {'offer id': 'offer_id'})
transcript.head()

,event,person,time,value,offer_or_amount,offer_id,amount,reward
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},offer,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},offer,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},offer,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},offer,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [14]:
transcript[transcript['person'] == '78afa995795e4d85b5d9ceeca43f5fef']

,event,person,time,value,offer_or_amount,offer_id,amount,reward
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
15561,offer viewed,78afa995795e4d85b5d9ceeca43f5fef,6,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
47582,transaction,78afa995795e4d85b5d9ceeca43f5fef,132,{'amount': 19.89},amount,NaN,19.89,NaN
47583,offer completed,78afa995795e4d85b5d9ceeca43f5fef,132,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,offer,NaN,NaN,5.0
49502,transaction,78afa995795e4d85b5d9ceeca43f5fef,144,{'amount': 17.78},amount,NaN,17.78,NaN
53176,offer received,78afa995795e4d85b5d9ceeca43f5fef,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN
85291,offer viewed,78afa995795e4d85b5d9ceeca43f5fef,216,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN
87134,transaction,78afa995795e4d85b5d9ceeca43f5fef,222,{'amount': 19.67},amount,NaN,19.67,NaN
92104,transaction,78afa995795e4d85b5d9ceeca43f5fef,240,{'amount': 29.72},amount,NaN,29.72,NaN
141566,transaction,78afa995795e4d85b5d9ceeca43f5fef,378,{'amount': 23.93},amount,NaN,23.93,NaN


In [15]:
transcript['event'].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'], dtype=object)

#### Sort Transcript by person and time

In [16]:
transcript=transcript.sort_values(['person','time'])

#### Create new dataframe that has the offer_ids and events listed out per person with respect to time

In [17]:
offers_view_transacted=transcript[['time','offer_id','person','event']][(transcript['event']=='transaction') | (transcript['event']=='offer viewed')].groupby(['person','offer_id']).ffill()
offers_view_transacted['offer_id']=offers_view_transacted['offer_id'].ffill()

In [18]:
offers_view_transacted

,person,offer_id,time,event
77705,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,192,offer viewed
89291,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,228,transaction
139992,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,372,offer viewed
168412,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,414,transaction
187554,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,456,offer viewed
228422,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,528,transaction
233413,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,540,offer viewed
237784,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,552,transaction
258883,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,576,transaction
293497,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,660,transaction


#### Next, I merge the data from the temporary df with our initial transcript data

In [19]:
transcript=transcript.merge(offers_view_transacted,how='left',on=['person','time','event'])

In [20]:
transcript.head()

,event,person,time,value,offer_or_amount,offer_id_x,amount,reward,offer_id_y
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16},amount,NaN,22.16,NaN,5a8bc65990b245e5a138643cd4eb9837
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'},offer,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'},offer,3f207df678b143eea3cee63160fa8bed,NaN,NaN,3f207df678b143eea3cee63160fa8bed


#### Now, I bring the two offer_id columns into one

In [21]:
transcript['offer_id']=np.where(transcript['offer_id_x'].isnull(),transcript['offer_id_y'],transcript['offer_id_x'])
transcript.drop(columns=['offer_id_x','offer_id_y'])

,event,person,time,value,offer_or_amount,amount,reward,offer_id
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16},amount,22.16,NaN,5a8bc65990b245e5a138643cd4eb9837
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'},offer,NaN,NaN,3f207df678b143eea3cee63160fa8bed
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'},offer,NaN,NaN,3f207df678b143eea3cee63160fa8bed
5,offer received,0009655768c64bdeb2e877511632db8f,408,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},offer,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d
6,transaction,0009655768c64bdeb2e877511632db8f,414,{'amount': 8.57},amount,8.57,NaN,3f207df678b143eea3cee63160fa8bed
7,offer completed,0009655768c64bdeb2e877511632db8f,414,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,offer,NaN,5.0,NaN
8,offer viewed,0009655768c64bdeb2e877511632db8f,456,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},offer,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d
9,offer received,0009655768c64bdeb2e877511632db8f,504,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},offer,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4


## Next, I merge transcript with portfolio

In [23]:
portfolio = portfolio.rename(index=str, columns={'id':'offer_id'})
transcript = transcript.merge(portfolio,how='left',on='offer_id')
transcript = transcript.drop(columns=['offer_id_x','offer_id_y', 'reward_x', 'offer_or_amount'])
transcript = transcript.rename(columns = {'reward_y':'reward'})
transcript.head()

,event,person,time,value,amount,offer_id,channels,difficulty,duration,offer_type,reward
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16},22.16,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0


### I examine the output for one person

In [24]:
transcript[transcript['person'] == '0009655768c64bdeb2e877511632db8f']

,event,person,time,value,amount,offer_id,channels,difficulty,duration,offer_type,reward
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16},22.16,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0
5,offer received,0009655768c64bdeb2e877511632db8f,408,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},NaN,f19421c1d4aa40978ebb69ca19b0e20d,"[web, email, mobile, social]",5.0,5.0,bogo,5.0
6,transaction,0009655768c64bdeb2e877511632db8f,414,{'amount': 8.57},8.57,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0
7,offer completed,0009655768c64bdeb2e877511632db8f,414,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,offer viewed,0009655768c64bdeb2e877511632db8f,456,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},NaN,f19421c1d4aa40978ebb69ca19b0e20d,"[web, email, mobile, social]",5.0,5.0,bogo,5.0
9,offer received,0009655768c64bdeb2e877511632db8f,504,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},NaN,fafdcd668e3743c1bb461111dcafc2a4,"[web, email, mobile, social]",10.0,10.0,discount,2.0


In [25]:
portfolio

,channels,difficulty,duration,offer_id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


## Next, I create a column defining how long a person has to redeem a given offer upon receipt of it.

In [35]:
time_avail_list = []
time_current_list = []
time_offer_duration_list = []

for idx in range(len(transcript)):
    
    time_current = (transcript['time'].loc[idx])
    
    if isinstance(transcript['offer_id'].loc[idx], str) == True:
        if transcript['event'].loc[idx] == 'offer received':
            time_offer_duration = (portfolio['duration'][portfolio['offer_id'] == transcript['offer_id'].loc[idx]].values[0]*24)
        else:
            time_offer_duration = np.nan
    else:
        time_offer_duration = np.nan
        
    #try:
    #    time_offer_duration = (portfolio['duration'][portfolio['offer_id'] == transcript['offer_id'].loc[idx]].values[0]*24)
    #except:
    time_avail = time_current + time_offer_duration
    #+ (portfolio['duration'][portfolio['offer_id'] == transcript['offer_id'].loc[idx]].values[0]*24)
    time_avail_list.append(time_avail)
    time_current_list.append(time_current)
    time_offer_duration_list.append(time_offer_duration)

In [36]:
transcript['time_to_redeem'] = time_avail_list

In [37]:
transcript.head()

,event,person,time,value,amount,offer_id,channels,difficulty,duration,offer_type,reward,time_to_redeem
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,240.0
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,NaN
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16},22.16,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,NaN
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0,432.0
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0,NaN


### I also create a column showing how long a given offer lasts upon receipt

In [38]:
transcript['duration_list'] = time_offer_duration_list
transcript['duration_list'][transcript['event'] != 'offer received'] = 0
transcript['duration_list'] = transcript['duration_list'].fillna(0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
transcript.head()

,event,person,time,value,amount,offer_id,channels,difficulty,duration,offer_type,reward,time_to_redeem,duration_list
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,240.0,72.0
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,NaN,0.0
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16},22.16,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,NaN,0.0
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0,432.0,96.0
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0,NaN,0.0


### Next, I clean time_to_redeem to get rid of null values

In [40]:
transcript['time_to_redeem'][1:][((transcript['person'] != transcript['person'].shift()) &
                                 (transcript['event'] != 'offer received'))] = 0
transcript['time_to_redeem'] = transcript['time_to_redeem'].ffill()

### Now, I create a column to keep track of how much overlap there was with two offers being run at once.  This becomes important later when trying to see how much a customer spent under the influence of each offer.

In [41]:
overlap_list = []

for idx in range(1,len(transcript)):
    
    if ((transcript['person'].loc[idx] == transcript['person'].loc[idx-1]) &
        (transcript['event'].loc[idx] == 'offer received') & 
        (transcript['time_to_redeem'].loc[idx-1] > transcript['time'].loc[idx])):
        overlap = transcript['time_to_redeem'].loc[idx-1] - transcript['time'].loc[idx]
    else:
        overlap = 0
        
    overlap_list.append(overlap)

overlap_list.insert(0,0)

In [43]:
transcript['overlap'] = overlap_list
transcript.head()

,event,person,time,value,amount,offer_id,channels,difficulty,duration,offer_type,reward,time_to_redeem,duration_list,overlap
0,offer received,0009655768c64bdeb2e877511632db8f,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,240.0,72.0,0.0
1,offer viewed,0009655768c64bdeb2e877511632db8f,192,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},NaN,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,240.0,0.0,0.0
2,transaction,0009655768c64bdeb2e877511632db8f,228,{'amount': 22.16},22.16,5a8bc65990b245e5a138643cd4eb9837,"[email, mobile, social]",0.0,3.0,informational,0.0,240.0,0.0,0.0
3,offer received,0009655768c64bdeb2e877511632db8f,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0,432.0,96.0,0.0
4,offer viewed,0009655768c64bdeb2e877511632db8f,372,{'offer id': '3f207df678b143eea3cee63160fa8bed'},NaN,3f207df678b143eea3cee63160fa8bed,"[web, email, mobile]",0.0,4.0,informational,0.0,432.0,0.0,0.0


### A final error handling to make sure all of our promotion times "end" when the dataset ends

In [44]:
transcript['duration_list'][((transcript['time_to_redeem'] > transcript['time'].max()) &
                           (transcript['duration_list'][transcript['event'] == 'offer received']))] = transcript['time'].max() - transcript['time']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## How many hours each person was advertised to

### First, we look at how much each person spent per offer_id

In [45]:
df_offers_given = transcript.groupby(['person', 'offer_id'])['amount'].sum().reset_index()
df_offers_given.head()

,person,offer_id,amount
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0.00
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,8.57
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,22.16
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,14.11
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,82.76


#### Next, we add duration per offer

In [47]:
df_offers_given = df_offers_given.merge(transcript.groupby(['person', 'offer_id'])['duration_list'].sum().reset_index())
df_offers_given.head(20)

,person,offer_id,amount,duration_list
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0.00,138.0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,8.57,96.0
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,22.16,72.0
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,14.11,120.0
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,82.76,210.0
5,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,4.09,240.0
6,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,240.0
7,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,11.93,168.0
8,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,13.49,96.0
9,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,0.00,72.0


#### Now, I add a new dataframe to keep track of how long each person was under the influence of offers as well as how much of the time multiple offers were influencing a customer together (overlap).  Additionally, I add how much time the person was not under the influence of any offer.

In [48]:
df_marketing_duration = df_offers_given.groupby(['person'])['duration_list'].sum().reset_index()
df_marketing_duration = df_marketing_duration.merge(transcript.groupby(['person'])['overlap'].sum().reset_index())
df_marketing_duration.head()

,person,duration_list,overlap
0,0009655768c64bdeb2e877511632db8f,636.0,216.0
1,00116118485d4dfda04fdbaba9a87b5c,240.0,0.0
2,0011e0d4e6b944f998e987f904e8c1e5,744.0,144.0
3,0020c2b971eb4e9188eac86d93036a77,840.0,264.0
4,0020ccbbb6d84e358d3414a3ff76cffd,528.0,48.0


In [49]:
df_marketing_duration['no_marketing_duration'] = transcript['time'].max() - df_marketing_duration['duration_list'] + df_marketing_duration['overlap']

In [50]:
df_marketing_duration.sample(20)

,person,duration_list,overlap,no_marketing_duration
359,058815f9957648d5ade6e9f8643e5e0a,810.0,192.0,96.0
13475,ca3ed3e47a1e4ceea8dcd8b2bbaf1076,600.0,0.0,114.0
14604,db74513485504a02ae8bc01be71e7ae6,864.0,264.0,114.0
12021,b3851741f02e4fc391f9d691cc1ce86b,210.0,0.0,504.0
6034,5aea02861ebc45f5992f8decd1086d38,552.0,120.0,282.0
2339,24ade653d9a4485c89f31a8316367930,882.0,168.0,0.0
1623,1929135b8300410d879b00b07ffe2012,600.0,48.0,162.0
6217,5d8afafc8f544d44a7c63cec59a17298,618.0,72.0,168.0
4766,476c7dc6eaa04c3b8a9ea0aa4ccc814e,192.0,0.0,522.0
439,06d34b0fb66e4feea9b2c765755a8e1f,714.0,0.0,0.0


#### Now, I make a dataframe to keep track of sales per customer when they weren't under the influence of any offers

In [54]:
df_non_offer_sales = transcript[transcript['time_to_redeem'] <= transcript['time']].groupby(['person'])['amount'].sum().reset_index()

#### This is added into the df_marketing_duration dataframe

In [55]:
df_marketing_duration = df_marketing_duration.merge(df_non_offer_sales[['person', 'amount']])
df_marketing_duration.head()

,person,duration_list,overlap,no_marketing_duration,amount
0,00116118485d4dfda04fdbaba9a87b5c,240.0,0.0,474.0,4.09
1,0011e0d4e6b944f998e987f904e8c1e5,744.0,144.0,114.0,13.49
2,0020c2b971eb4e9188eac86d93036a77,840.0,264.0,138.0,47.43
3,0020ccbbb6d84e358d3414a3ff76cffd,528.0,48.0,234.0,23.74
4,003d66b6608740288d6cc97a6903f4f0,858.0,240.0,96.0,2.07


#### Create a non-offer revenue rate column

In [56]:
df_marketing_duration['non_offer_rev_rate'] = df_marketing_duration['amount'] / df_marketing_duration['no_marketing_duration']
df_marketing_duration.head()

,person,duration_list,overlap,no_marketing_duration,amount,non_offer_rev_rate
0,00116118485d4dfda04fdbaba9a87b5c,240.0,0.0,474.0,4.09,0.008629
1,0011e0d4e6b944f998e987f904e8c1e5,744.0,144.0,114.0,13.49,0.118333
2,0020c2b971eb4e9188eac86d93036a77,840.0,264.0,138.0,47.43,0.343696
3,0020ccbbb6d84e358d3414a3ff76cffd,528.0,48.0,234.0,23.74,0.101453
4,003d66b6608740288d6cc97a6903f4f0,858.0,240.0,96.0,2.07,0.021562


#### Create an offer revenue rate column

In [57]:
df_offers_given['offer_rev_rate'] = df_offers_given['amount'] / df_offers_given['duration_list']
df_offers_given.head()

,person,offer_id,amount,duration_list,offer_rev_rate
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0.00,138.0,0.000000
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,8.57,96.0,0.089271
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,22.16,72.0,0.307778
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,14.11,120.0,0.117583
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,82.76,210.0,0.394095


#### Add in non-offer revenue rate to the revenue rate per offer dataframe and create column showing incremental revenue rate

In [58]:
df_offers_given = df_offers_given.merge(df_marketing_duration[['person', 'non_offer_rev_rate']])
df_offers_given['incr_rev_rate'] = df_offers_given['offer_rev_rate'] - df_offers_given['non_offer_rev_rate']
df_offers_given.sample(10)

,person,offer_id,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate
8636,363c28b57ad34b468e3a687fd904b6c1,4d5c57ea9a6940dd891ad53e9dbe8da0,18.81,120.0,0.156750,0.028542,0.128208
32066,c773fe9f23f04e9ba2c70ad5e237c7db,ae264e3637204a6fb9bb56bc8210ddfd,36.66,138.0,0.265652,0.169187,0.096465
10938,43aae84d54144c1bacf027bbb00f4ffa,f19421c1d4aa40978ebb69ca19b0e20d,47.12,240.0,0.196333,0.026746,0.169587
37308,e7f21a3d28164d98aad6595510521f8e,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,240.0,0.000000,0.090504,-0.090504
31241,c20df4c48430421fb2b738cce8ae8f71,3f207df678b143eea3cee63160fa8bed,7.20,96.0,0.075000,0.033915,0.041085
39460,f58e8897221343baa70ee46717883176,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,168.0,0.000000,0.008571,-0.008571
25535,9df2790f060d430e94f17189042e5671,fafdcd668e3743c1bb461111dcafc2a4,45.93,210.0,0.218714,0.077396,0.141318
40737,fddb6eacd12444ceb6ec5c9e301f4d79,2906b810c7d4411798c6938adc9daaa5,110.25,168.0,0.656250,0.230119,0.426131
30250,bb272cd4cac24fcc8f0168cfb5c6f063,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,168.0,0.000000,0.052174,-0.052174
30614,bda8280eee0249cb9a4b282a993aaafd,2298d6c36e964ae4a3e7e9706d1fb8c2,176.87,168.0,1.052798,0.193856,0.858941


### Here, I have to drop 'inf' values in df_offers_given
#### This is because these offers weren't assigned an 'offer received' timestamp, so they had no official duration timeline
#### This loses around 3,000 rows

In [61]:
df_offers_given.shape

(41017, 7)

In [62]:
df_offers_given = df_offers_given.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [63]:
df_offers_given.shape

(37915, 7)

In [64]:
df_offers_given.head(10)

,person,offer_id,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate
0,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,4.09,240.0,0.017042,0.008629,0.008413
1,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,240.0,0.000000,0.118333,-0.118333
2,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,11.93,168.0,0.071012,0.118333,-0.047321
3,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,13.49,96.0,0.140521,0.118333,0.022188
4,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,0.00,72.0,0.000000,0.118333,-0.118333
5,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,54.04,168.0,0.321667,0.118333,0.203333
6,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,51.10,120.0,0.425833,0.343696,0.082138
7,0020c2b971eb4e9188eac86d93036a77,5a8bc65990b245e5a138643cd4eb9837,47.43,72.0,0.658750,0.343696,0.315054
8,0020c2b971eb4e9188eac86d93036a77,ae264e3637204a6fb9bb56bc8210ddfd,0.00,168.0,0.000000,0.343696,-0.343696
9,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,98.33,480.0,0.204854,0.343696,-0.138841


### Examine data for nulls and other weirdness

In [65]:
profile[profile['age'] == 118]

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN
6,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN
7,118,20171002,None,68617ca6246f4fbc85e91a2a49552598,NaN
9,118,20161122,None,8974fc5686fe429db53ddde067b88302,NaN
10,118,20170824,None,c4863c7985cf408faee930f111475da3,NaN
11,118,20150919,None,148adfcaa27d485b82f323aaaad036bd,NaN
17,118,20170801,None,744d603ef08c4f33af5a61c8c7628d1c,NaN
23,118,20170907,None,2b826eba31074a059d63b0ae8f50b7d5,NaN


#### People aged 118 have no gender or income.  My analysis will be heavily dependent on age, gender, and income, so I'll drop these values because they only represent 13% of the total data set.

In [66]:
profile.dropna().shape[0] / profile.shape[0]

0.8720588235294118

In [67]:
profile_no_nan = profile.dropna()
df_offers_given = df_offers_given.merge(profile_no_nan, left_on='person', right_on='id')
df_offers_given = df_offers_given.drop(columns='id')
df_offers_given.head(10)

,person,offer_id,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate,age,became_member_on,gender,income
0,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,240.0,0.000000,0.118333,-0.118333,40,20180109,O,57000.0
1,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,11.93,168.0,0.071012,0.118333,-0.047321,40,20180109,O,57000.0
2,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,13.49,96.0,0.140521,0.118333,0.022188,40,20180109,O,57000.0
3,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,0.00,72.0,0.000000,0.118333,-0.118333,40,20180109,O,57000.0
4,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,54.04,168.0,0.321667,0.118333,0.203333,40,20180109,O,57000.0
5,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,51.10,120.0,0.425833,0.343696,0.082138,59,20160304,F,90000.0
6,0020c2b971eb4e9188eac86d93036a77,5a8bc65990b245e5a138643cd4eb9837,47.43,72.0,0.658750,0.343696,0.315054,59,20160304,F,90000.0
7,0020c2b971eb4e9188eac86d93036a77,ae264e3637204a6fb9bb56bc8210ddfd,0.00,168.0,0.000000,0.343696,-0.343696,59,20160304,F,90000.0
8,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,98.33,480.0,0.204854,0.343696,-0.138841,59,20160304,F,90000.0
9,0020ccbbb6d84e358d3414a3ff76cffd,2298d6c36e964ae4a3e7e9706d1fb8c2,46.59,168.0,0.277321,0.101453,0.175868,24,20161111,F,60000.0


In [68]:
df_offers_given.describe()

,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate,age,became_member_on,income
count,33290.000000,33290.000000,33290.000000,33290.000000,33290.000000,33290.000000,3.329000e+04,33290.000000
mean,31.529988,172.734935,0.198976,0.133736,0.065240,53.847942,2.016616e+07,64224.842295
std,60.765803,83.857265,0.379296,0.284298,0.429250,17.586178,1.209456e+04,21405.546576
min,0.000000,72.000000,0.000000,0.000000,-13.420625,18.000000,2.013073e+07,30000.000000
25%,0.000000,120.000000,0.000000,0.030703,-0.060583,42.000000,2.016042e+07,48000.000000
50%,14.270000,168.000000,0.090595,0.085699,0.010333,55.000000,2.017062e+07,62000.000000
75%,42.090000,210.000000,0.267655,0.170476,0.139954,66.000000,2.017122e+07,78000.000000
max,1136.780000,930.000000,10.453229,13.420625,8.412851,101.000000,2.018073e+07,120000.000000


#### Convert became_member_on to number of days since becoming a member

In [69]:
df_offers_given['became_member_on'] = pd.to_datetime(df_offers_given['became_member_on'], format='%Y%m%d', errors='ignore')

def get_days_member(x):
    
    y = (df_offers_given['became_member_on'].max() - x).total_seconds() / 86400
    
    return y

df_offers_given['days_member'] = df_offers_given['became_member_on'].apply(lambda x: get_days_member(x))
df_offers_given.head(10)

,person,offer_id,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate,age,became_member_on,gender,income,days_member
0,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,240.0,0.000000,0.118333,-0.118333,40,2018-01-09,O,57000.0,198.0
1,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,11.93,168.0,0.071012,0.118333,-0.047321,40,2018-01-09,O,57000.0,198.0
2,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,13.49,96.0,0.140521,0.118333,0.022188,40,2018-01-09,O,57000.0,198.0
3,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,0.00,72.0,0.000000,0.118333,-0.118333,40,2018-01-09,O,57000.0,198.0
4,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,54.04,168.0,0.321667,0.118333,0.203333,40,2018-01-09,O,57000.0,198.0
5,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,51.10,120.0,0.425833,0.343696,0.082138,59,2016-03-04,F,90000.0,874.0
6,0020c2b971eb4e9188eac86d93036a77,5a8bc65990b245e5a138643cd4eb9837,47.43,72.0,0.658750,0.343696,0.315054,59,2016-03-04,F,90000.0,874.0
7,0020c2b971eb4e9188eac86d93036a77,ae264e3637204a6fb9bb56bc8210ddfd,0.00,168.0,0.000000,0.343696,-0.343696,59,2016-03-04,F,90000.0,874.0
8,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,98.33,480.0,0.204854,0.343696,-0.138841,59,2016-03-04,F,90000.0,874.0
9,0020ccbbb6d84e358d3414a3ff76cffd,2298d6c36e964ae4a3e7e9706d1fb8c2,46.59,168.0,0.277321,0.101453,0.175868,24,2016-11-11,F,60000.0,622.0


# Prepare data for modeling

### Make dummies for offer_id

In [70]:
df_offers_given['offer_id'].unique()

array(['0b1e1539f2cc45b7b9fa7c272da2e1d7',
       '2298d6c36e964ae4a3e7e9706d1fb8c2',
       '3f207df678b143eea3cee63160fa8bed',
       '5a8bc65990b245e5a138643cd4eb9837',
       '9b98b8c7a33c4b65b9aebfe6a799e6d9',
       '4d5c57ea9a6940dd891ad53e9dbe8da0',
       'ae264e3637204a6fb9bb56bc8210ddfd',
       'fafdcd668e3743c1bb461111dcafc2a4',
       'f19421c1d4aa40978ebb69ca19b0e20d',
       '2906b810c7d4411798c6938adc9daaa5'], dtype=object)

In [71]:
df_offers_given = pd.concat([df_offers_given, pd.get_dummies(data = df_offers_given['offer_id'], prefix = 'offer_id')], axis = 1)

In [72]:
df_offers_given[['offer_id', 'offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7', 'offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2']].sample(10)

,offer_id,offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2
9972,fafdcd668e3743c1bb461111dcafc2a4,0,0
2370,2906b810c7d4411798c6938adc9daaa5,0,0
32361,5a8bc65990b245e5a138643cd4eb9837,0,0
18569,5a8bc65990b245e5a138643cd4eb9837,0,0
6721,3f207df678b143eea3cee63160fa8bed,0,0
12632,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1
19203,ae264e3637204a6fb9bb56bc8210ddfd,0,0
12222,fafdcd668e3743c1bb461111dcafc2a4,0,0
3948,fafdcd668e3743c1bb461111dcafc2a4,0,0
32260,3f207df678b143eea3cee63160fa8bed,0,0


### Make dummies for gender

In [73]:
df_offers_given['gender'].unique()

array(['O', 'F', 'M'], dtype=object)

In [74]:
df_offers_given = pd.concat([df_offers_given, pd.get_dummies(data = df_offers_given['gender'], prefix = 'gender')], axis = 1)

In [75]:
df_offers_given[['gender', 'gender_O', 'gender_F', 'gender_M']].sample(10)

,gender,gender_O,gender_F,gender_M
13320,F,0,1,0
3017,F,0,1,0
5307,M,0,0,1
7088,M,0,0,1
23018,M,0,0,1
821,F,0,1,0
7370,F,0,1,0
1700,M,0,0,1
9314,M,0,0,1
32911,F,0,1,0


#### Final columns for modeling

In [76]:
df_offers_given.columns

Index(['person', 'offer_id', 'amount', 'duration_list', 'offer_rev_rate',
       'non_offer_rev_rate', 'incr_rev_rate', 'age', 'became_member_on',
       'gender', 'income', 'days_member',
       'offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7',
       'offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2',
       'offer_id_2906b810c7d4411798c6938adc9daaa5',
       'offer_id_3f207df678b143eea3cee63160fa8bed',
       'offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0',
       'offer_id_5a8bc65990b245e5a138643cd4eb9837',
       'offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9',
       'offer_id_ae264e3637204a6fb9bb56bc8210ddfd',
       'offer_id_f19421c1d4aa40978ebb69ca19b0e20d',
       'offer_id_fafdcd668e3743c1bb461111dcafc2a4', 'gender_F', 'gender_M',
       'gender_O'],
      dtype='object')

# Model 1 - Typical train/test split
## With all of the data cleaned, I first look at a basic train/test split where data is split into training and testing groups at random.
### Set up training and testing sets

In [77]:
X = df_offers_given.drop(columns = ['person', 'offer_id', 'amount', 'duration_list', 'offer_rev_rate',
       'non_offer_rev_rate','became_member_on', 'gender'])

In [78]:
Y = df_offers_given['incr_rev_rate']

In [79]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

### Prep Pipeline

In [80]:
def build_model():
    """Build ML model using Pipeline"""
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestRegressor())
    ])
    
    parameters = {
        'clf__n_estimators': (50, 100, 200, 400)
    }
    
    cv = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=-1, cv=KFold(n_splits=5, shuffle=True,
                                                                                              random_state=7))
    
    return cv

### Build, Train, and Test Model

In [81]:
start_time = time.time()

print('Building model...')
model = build_model()

print('Training model...')
model_result = model.fit(X_train, Y_train)

best_model = model_result.best_estimator_

y_pred = best_model.predict(X_test)

print("--- Model --- %.2f minutes ---" % ((time.time() - start_time) / 60))
#print("--- Model --- %s tests ---" % (len(param_grid)))
#print("--- Model --- Best: using %s" % (best_model.best_params_))
print("--- Model --- Root Mean Squared Error")
print((mean_squared_error(Y_test, y_pred))**(1/2))

Building model...
Training model...
--- Model --- 4.96 minutes ---
--- Model --- Root Mean Squared Error
0.035705721746


### Look more closely at error

In [83]:
df_test = X_test.copy()

In [84]:
df_test['incr_rev_rate'] = Y_test

In [85]:
df_test['y_pred'] = y_pred

In [86]:
df_test['Error'] = abs(df_test['y_pred'] - df_test['incr_rev_rate'])

In [87]:
df_test[['incr_rev_rate', 'y_pred', 'Error']].describe()

,incr_rev_rate,y_pred,Error
count,6658.000000,6658.000000,6.658000e+03
mean,0.058917,0.059517,8.877031e-04
std,0.419314,0.401523,3.569737e-02
min,-13.191075,-11.133620,4.579053e-10
25%,-0.062973,-0.062943,2.234404e-05
50%,0.008838,0.008836,5.027659e-05
75%,0.137800,0.137780,9.401589e-05
max,6.638277,6.660961,2.060329e+00


# Model 2 - Group by customer so that our testing is on customers the model hasn't seen before
## This will help us see how the model will generalize to new customers because all of the customers in the testing set are new to the model
### Set up training and testing sets

### Create training and testing sets based on number of individuals in each
### Still use 80% of data as training and 20% as testing

In [88]:
df_offers_given.shape[0]

33290

In [89]:
df_offers_given.shape[0]*0.8

26632.0

In [90]:
df_offers_given[26632:26636]

,person,offer_id,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate,age,became_member_on,gender,...,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,gender_F,gender_M,gender_O
26632,cc8812db238a4bb9962dbef576925c5b,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,210.0,0.000000,0.190899,-0.190899,40,2015-09-03,F,...,0,0,0,0,0,0,0,1,0,0
26633,cc8812db238a4bb9962dbef576925c5b,ae264e3637204a6fb9bb56bc8210ddfd,28.17,138.0,0.204130,0.190899,0.013232,40,2015-09-03,F,...,0,0,0,0,1,0,0,1,0,0
26634,cc884499447d4181b59c89efb6e71d84,2298d6c36e964ae4a3e7e9706d1fb8c2,14.32,168.0,0.085238,0.041714,0.043524,28,2015-02-18,M,...,0,0,0,0,0,0,0,0,1,0
26635,cc884499447d4181b59c89efb6e71d84,2906b810c7d4411798c6938adc9daaa5,0.00,168.0,0.000000,0.041714,-0.041714,28,2015-02-18,M,...,0,0,0,0,0,0,0,0,1,0


### Index 26634 seems like a good last row for training data.  It's roughly 80% of the full dataset and cleanly finishes an individual's data

In [91]:
df_offers_given[:26634].tail()

,person,offer_id,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate,age,became_member_on,gender,...,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,gender_F,gender_M,gender_O
26629,cc82f3d926344efa819d55e0e6b3ecef,5a8bc65990b245e5a138643cd4eb9837,14.79,72.0,0.205417,0.083102,0.122315,64,2016-06-05,F,...,0,0,1,0,0,0,0,1,0,0
26630,cc82f3d926344efa819d55e0e6b3ecef,9b98b8c7a33c4b65b9aebfe6a799e6d9,26.81,168.0,0.159583,0.083102,0.076481,64,2016-06-05,F,...,0,0,0,1,0,0,0,1,0,0
26631,cc82f3d926344efa819d55e0e6b3ecef,f19421c1d4aa40978ebb69ca19b0e20d,43.97,120.0,0.366417,0.083102,0.283315,64,2016-06-05,F,...,0,0,0,0,0,1,0,1,0,0
26632,cc8812db238a4bb9962dbef576925c5b,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,210.0,0.000000,0.190899,-0.190899,40,2015-09-03,F,...,0,0,0,0,0,0,0,1,0,0
26633,cc8812db238a4bb9962dbef576925c5b,ae264e3637204a6fb9bb56bc8210ddfd,28.17,138.0,0.204130,0.190899,0.013232,40,2015-09-03,F,...,0,0,0,0,1,0,0,1,0,0


In [92]:
df_offers_given[26634:].head()

,person,offer_id,amount,duration_list,offer_rev_rate,non_offer_rev_rate,incr_rev_rate,age,became_member_on,gender,...,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,gender_F,gender_M,gender_O
26634,cc884499447d4181b59c89efb6e71d84,2298d6c36e964ae4a3e7e9706d1fb8c2,14.32,168.0,0.085238,0.041714,0.043524,28,2015-02-18,M,...,0,0,0,0,0,0,0,0,1,0
26635,cc884499447d4181b59c89efb6e71d84,2906b810c7d4411798c6938adc9daaa5,0.00,168.0,0.000000,0.041714,-0.041714,28,2015-02-18,M,...,0,0,0,0,0,0,0,0,1,0
26636,cc884499447d4181b59c89efb6e71d84,ae264e3637204a6fb9bb56bc8210ddfd,120.13,168.0,0.715060,0.041714,0.673345,28,2015-02-18,M,...,0,0,0,0,1,0,0,0,1,0
26637,cc884499447d4181b59c89efb6e71d84,f19421c1d4aa40978ebb69ca19b0e20d,0.00,120.0,0.000000,0.041714,-0.041714,28,2015-02-18,M,...,0,0,0,0,0,1,0,0,1,0
26638,cc8ddacae6e84c6e81f19c2b70306b6f,2298d6c36e964ae4a3e7e9706d1fb8c2,116.73,168.0,0.694821,0.126190,0.568631,36,2017-10-09,F,...,0,0,0,0,0,0,0,1,0,0


In [93]:
df_train = df_offers_given[:26634]
df_test = df_offers_given[26634:]

In [94]:
train_groups = np.asarray(df_train['person'])

In [95]:
gss = GroupShuffleSplit(n_splits = 10, test_size = 0.2, random_state=12)

In [96]:
X_train = df_train.drop(columns = ['person', 'offer_id', 'amount', 'duration_list', 'offer_rev_rate',
       'non_offer_rev_rate','became_member_on', 'gender'])

X_test = df_test.drop(columns = ['person', 'offer_id', 'amount', 'duration_list', 'offer_rev_rate',
       'non_offer_rev_rate','became_member_on', 'gender'])

Y_train = df_train['incr_rev_rate']

Y_test = df_test['incr_rev_rate']

In [97]:
gss.split(X_train, Y_train, groups=train_groups)
gss.get_n_splits(X_train, Y_train, train_groups)

10

### Prep Pipeline

In [98]:
def build_model():
    """Build ML model using Pipeline"""
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestRegressor())
    ])
    
    parameters = {
        'clf__n_estimators': (50, 100, 200, 400)
    }
    
    cv = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=-1, cv=gss)
    
    return cv

In [99]:
start_time = time.time()

print('Building model...')
model = build_model()

print('Training model...')
model_result = model.fit(X_train, Y_train, groups = train_groups)

best_model = model_result.best_estimator_

y_pred = best_model.predict(X_test)

print("--- Model --- %.2f minutes ---" % ((time.time() - start_time) / 60))
#print("--- Model --- %s tests ---" % (len(param_grid)))
#print("--- Model --- Best: using %s" % (best_model.best_params_))
print("--- Model --- Root Mean Squared Error")
print((mean_squared_error(Y_test, y_pred))**(1/2))

Building model...
Training model...
--- Model --- 9.54 minutes ---
--- Model --- Root Mean Squared Error
0.145100278182


In [100]:
Y_test.mean()

0.066108894413503816

In [101]:
df_test = X_test.copy()

In [102]:
df_test['incr_rev_rate'] = Y_test

In [103]:
df_test['y_pred'] = y_pred

In [104]:
df_test['Error'] = abs(df_test['y_pred'] - df_test['incr_rev_rate'])

In [105]:
df_test[['incr_rev_rate', 'y_pred', 'Error']].describe()

,incr_rev_rate,y_pred,Error
count,6656.000000,6656.000000,6.656000e+03
mean,0.066109,0.069827,3.964851e-03
std,0.497267,0.416798,1.450570e-01
min,-13.420625,-7.278645,1.608918e-09
25%,-0.056704,-0.056750,2.262096e-05
50%,0.013779,0.013758,5.316404e-05
75%,0.146125,0.146270,9.961676e-05
max,6.337343,6.459845,6.199890e+00


In [106]:
df_test['Error'].quantile(0.95)

0.0002972615490607483

# Conclusions

##  Findings
### In the end, both models performed well.  The first model only had a median error of 0.00005 and a RMSE of 0.0357 while the second had a median error of 0.000053 and a RMSE of 0.1451.  This is impressive given that the median of the value predicted (incremental revenue per hour) was 0.013779.
### Despite the fact the second model performed worse on testing data, it may be preferable to the first because it was trained and tested with separate customer groups.  This second model's testing seems more likely to perform similar to how the model would on real, new customers.
### This model should be able to be used going forward as a way to accurately predict which customers will spend more money upon receiving an offer and how much they will spend per hour of being promoted to.
##  Improvement
### Not much time was dedicated to optimizing hyperparameters, so there is likely room for improvement there.  Additionally, more data is always welcome, and the model is only trained for use when all demographic components are known for a given customer.
### In a subset of the customers, no age, gender, or income information was supplied.  This model will be unable to predict well on that subset because it assumes those variables are included.  Further research should be done to find best offers for that subset.